# Handling Data over time

There's a widespread trend in solar physics at the moment for correlation over actual science, so being able to handle data over time spans is a skill we all need to have. Python has ample support for this so lets have a look at wjat we can use.

So lets import some data to handle:

In [2]:
import numpy as np
data = np.loadtxt(fname='macrospicules', delimiter=',')

ValueError: Wrong number of columns at line 3